In [99]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
#import lightgbm as lgbm
#from xgboost import XGBRegressor
import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc

In [100]:
sys.path.append(os.path.realpath('..'))

In [63]:
train_df = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\train.csv")
test_df = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\test.csv")
sample_sub = pd.read_csv(r"D:\source\repos\Kaggle_Tabular_Playground_Series-ML\Jan-2023-S2\data\sample_submission.csv")

In [64]:
columns_to_vectorize = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
for vector_target in columns_to_vectorize:
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

In [65]:
features = ['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'gender_v', 'ever_married_v', 'work_type_v', 'Residence_type_v', 'smoking_status_v']
target = ['stroke']

In [66]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [67]:
y_min = Y.min()
y_max = Y.max()

print(y_min, y_max)

def my_rmse(y_true, y_hat):
    y_true[y_true < y_min] = y_min
    y_true[y_true > y_max] = y_max
    
    y_hat[y_hat < y_min] = y_min
    y_hat[y_hat > y_max] = y_max
    
    y_true_nan = np.isnan(y_true)
    y_hat_nan = np.isnan(y_hat)
    
    if y_true_nan.sum() > 0:
        print(y_true_nan.sum())
        np.where(y_true_nan, np.ma.array(y_true, mask=np.isnan(y_true)).mean(axis=0), y_true)
    if y_hat_nan.sum() > 0:
        print(y_hat_nan.sum())
        np.where(y_hat_nan, np.ma.array(y_hat, mask=np.isnan(y_hat)).mean(axis=0), y_hat)
    
    return mean_squared_error(y_true, y_hat, squared=False)

0.0 1.0


In [78]:
def xgb_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.4)

    param_grid = {
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'n_estimators': trial.suggest_int('n_estimators', 2, 100, 1), 
        'eta': trial.suggest_float('eta', 0.0007, 0.113), 
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 40), 

    } 
    
    reg = xgb.XGBModel(
        # These parameters should help with trial speed.
        objective='binary:logistic',
        tree_method='gpu_hist',
        booster='gbtree',
        predictor='gpu_predictor',
        n_jobs=4,
        eval_metric='auc',
        **param_grid
    )
    
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    
    preds = reg.predict(X_valid)
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    xgb_ranks[roc_auc] = reg

    print(roc_auc)
    return roc_auc

In [82]:
xgb_ranks = {}
    
train_time = 1 * 60 * 60
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='XGBRegressor')
study.optimize(xgb_objective, timeout=train_time)

[I 2023-01-16 08:10:56,891] A new study created in memory with name: XGBRegressor
[I 2023-01-16 08:10:57,432] Trial 0 finished with value: 0.8671208742660066 and parameters: {'max_depth': 15, 'n_estimators': 54, 'eta': 0.03126773288375842, 'subsample': 0.8982141684871907, 'colsample_bytree': 0.6096844135901044, 'reg_lambda': 33.168786195486796}. Best is trial 0 with value: 0.8671208742660066.


0.8671208742660066


[I 2023-01-16 08:10:57,813] Trial 1 finished with value: 0.8798261381399017 and parameters: {'max_depth': 11, 'n_estimators': 75, 'eta': 0.04752160474712603, 'subsample': 0.10230443889190463, 'colsample_bytree': 0.31874602438285665, 'reg_lambda': 29.14099640897655}. Best is trial 1 with value: 0.8798261381399017.


0.8798261381399017


[I 2023-01-16 08:10:58,159] Trial 2 finished with value: 0.8038572064515304 and parameters: {'max_depth': 18, 'n_estimators': 73, 'eta': 0.005540107770811843, 'subsample': 0.6440882070917302, 'colsample_bytree': 0.32409071126049693, 'reg_lambda': 19.1982675274905}. Best is trial 1 with value: 0.8798261381399017.


...

0.8744953217717262


[I 2023-01-16 08:13:58,711] Trial 434 finished with value: 0.887204626438775 and parameters: {'max_depth': 6, 'n_estimators': 84, 'eta': 0.04243273975138867, 'subsample': 0.5590749816421332, 'colsample_bytree': 0.78978923883752, 'reg_lambda': 15.614402730270045}. Best is trial 352 with value: 0.9035597367288568.


0.887204626438775


[W 2023-01-16 08:13:59,000] Trial 435 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_43632\1290881014.py", line 26, in xgb_objective
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\sklearn.py", line 1025, in fit
    self._Booster = train(
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\training.py", line 185, in train
    bst.update(dtrain, i, obj)
  File "D:\source\repos\venv\Python310\lib\site-packa

KeyboardInterrupt: 

In [83]:
xgb_ranks_bytes = pickle.dumps(xgb_ranks)
with open("xgb_ranks_bytes.hex", "wb") as binary_file:
    binary_file.write(xgb_ranks_bytes)

In [98]:
percent = math.ceil(len(xgb_ranks.keys()) * .1)
top = sorted(list(xgb_ranks.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    train_preds.append(xgb_ranks[key].predict(X))
    test_preds.append(xgb_ranks[key].predict(X_test))
    
train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 51.04it/s]


In [102]:
XGB_train_submission = pd.DataFrame(data={'id': train_df.id, 'stroke': train_final_preds})
XGB_train_submission.to_csv(fr'rendered_data/XGB_train_submission.csv', index=False)

XGB_test_submission = pd.DataFrame(data={'id': test_df.id, 'stroke': test_final_preds})
XGB_test_submission.to_csv(fr'rendered_data/XGB_test_submission.csv', index=False)